In [1]:
import random
import os
import json
import string
import hvac

from tfe.core.session import TFESession
from tfe.core.organization import Organization
from tfe.core.workspace import Workspace, VCSRepo
from tfe.core.oauth_client import OauthClient


def personal_access_token():
    client = hvac.Client(url=os.environ.get('VAULT_ADDR'), token=os.environ.get("VAULT_TOKEN"))
    client = hvac.Client(
        url=os.environ.get('VAULT_ADDR'), 
        token=os.environ.get("VAULT_TOKEN")
    )
    return client.read("secret/credentials/github").get("data").get("personal_access_token")


timeout = random.randint(100,600)
rand_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=24))
session = TFESession("https://app.terraform.io", os.environ.get("ATLAS_TOKEN"))
org = Organization("Darnold-PyTFE")
org.logger.setLevel("ERROR")
org.email = "darnold@hashicorp.com"
org.session_remember = timeout
org.session_timeout = timeout
org.collaborator_auth_policy = "two_factor_mandatory"
org.create()


oauth = OauthClient()
oauth.logger.setLevel("DEBUG")
oauth.organization = org
oauth.service_provider = "github"
oauth.personal_access_token = personal_access_token()
oauth.http_url = "https://github.com"
oauth.api_url = "https://api.github.com"
oauth.logger.debug(
    "Oauth Create: {0}".format(str(oauth.create().id))
)

vcs_repo = VCSRepo()
vcs_repo.identifier = "HappyPathway/aws-network-hosts"
vcs_repo.oauth_token_id = oauth.token
vcs_repo.branch = "master"

ws = Workspace()
ws.logger.setLevel("DEBUG")
ws.organization = org
ws.vcs_repo = vcs_repo
ws.name = "AWSNetworks"
ws.logger.debug(
    "Workspace Create: {0}".format(str(ws.create()))
)
print(
    ws.render()
)
ws2 = Workspace(ws.id, organization=org, name="AWSNetworks{0}".format(rand_string))
assert(ws2.vcs_repo.identifier == "HappyPathway/aws-network-hosts")

org.delete()


2018-10-12 23:02:03,799 com.happypathway.tfe.Organization: 404 Client Error: Not Found for url: https://app.terraform.io/api/v2/organizations/Darnold-2OA5Z7SWS9CEW4JW7BUNZLRS
2018-10-12 23:02:03,801 com.happypathway.tfe.Organization: 'NoneType' object has no attribute 'get'
2018-10-12 23:02:04,435 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: name
2018-10-12 23:02:04,436 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: created-at
2018-10-12 23:02:04,436 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: session-timeout
2018-10-12 23:02:04,437 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: session-remember
2018-10-12 23:02:04,439 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: collaborator-auth-policy
2018-10-12 23:02:04,440 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: enterpris

HTTPError: 422 Client Error: Unprocessable Entity for url: https://app.terraform.io/api/v2/organizations/Darnold-2OA5Z7SWS9CEW4JW7BUNZLRS/workspaces

In [28]:
import random
import os
import json
import string
import time
import hvac

from tfe.session import TFESession
from tfe.organization import Organization
from tfe.workspace import Workspace, VCSRepo
from tfe.oauth_client import OauthClient
from tfe.configuration import Configuration

session = TFESession("https://app.terraform.io", os.environ.get("ATLAS_TOKEN"))
ws = Workspace(organization="Darnold-Hashicorp", name="VaultPolicies")
ws.current_run.get()

conf = Configuration(workspace=ws)
for _c in conf.list():
    _c.get()
    print(
        json.dumps(
            _c.raw,
            separators=(',', ':'),
            indent=4,
            sort_keys=True
        )
    )
    break

run = ws.current_run
run.run_message = "Running From Python"
run.workspace_id = ws.id
run.configuration_version = "cv-Uf5DGk5EyzFwykDi"
run.create()
run.get()

while run.status not in ["errored"]:
    if run.status == "planned":
        run.apply()
    time.sleep(3)
    run.get()


2018-10-12 23:26:48,726 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: name
2018-10-12 23:26:48,727 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: created-at
2018-10-12 23:26:48,735 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: session-timeout
2018-10-12 23:26:48,737 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: session-remember
2018-10-12 23:26:48,738 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: collaborator-auth-policy
2018-10-12 23:26:48,740 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: enterprise-plan
2018-10-12 23:26:48,741 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: permissions
2018-10-12 23:26:48,742 com.happypathway.tfe.Organization: OrganizationValidator: does not have attribute: trial-expires-at
2018-10-12 23:26:48,744 c

{
    "data":{
        "attributes":{
            "auto-queue-runs":true,
            "error":null,
            "error-message":null,
            "source":"github",
            "status":"uploaded",
            "status-timestamps":{}
        },
        "id":"cv-Uf5DGk5EyzFwykDi",
        "links":{
            "self":"/api/v2/configuration-versions/cv-Uf5DGk5EyzFwykDi"
        },
        "relationships":{
            "ingress-attributes":{
                "data":{
                    "id":"ia-G4mzGdi1kA38rKAz",
                    "type":"ingress-attributes"
                },
                "links":{
                    "related":"/api/v2/configuration-versions/cv-Uf5DGk5EyzFwykDi/ingress-attributes"
                }
            }
        },
        "type":"configuration-versions"
    }
}


2018-10-12 23:26:51,315 com.happypathway.tfe.Run: RunValidator: does not have attribute: is-destroy
2018-10-12 23:26:51,317 com.happypathway.tfe.Run: RunValidator: does not have attribute: message
2018-10-12 23:26:51,319 com.happypathway.tfe.Run: RunValidator: does not have attribute: source
2018-10-12 23:26:51,320 com.happypathway.tfe.Run: can't set attribute
2018-10-12 23:26:51,321 com.happypathway.tfe.Run: RunValidator: does not have attribute: status-timestamps
2018-10-12 23:26:51,322 com.happypathway.tfe.Run: RunValidator: does not have attribute: created-at
2018-10-12 23:26:51,323 com.happypathway.tfe.Run: RunValidator: does not have attribute: canceled-at
2018-10-12 23:26:51,324 com.happypathway.tfe.Run: RunValidator: does not have attribute: has-changes
2018-10-12 23:26:51,324 com.happypathway.tfe.Run: RunValidator: does not have attribute: actions
2018-10-12 23:26:51,325 com.happypathway.tfe.Run: RunValidator: does not have attribute: plan-only
2018-10-12 23:26:51,326 com.happ

2018-10-12 23:27:19,446 com.happypathway.tfe.Run: RunValidator: does not have attribute: actions
2018-10-12 23:27:19,447 com.happypathway.tfe.Run: RunValidator: does not have attribute: plan-only
2018-10-12 23:27:19,447 com.happypathway.tfe.Run: RunValidator: does not have attribute: permissions
2018-10-12 23:27:23,279 com.happypathway.tfe.Run: RunValidator: does not have attribute: is-destroy
2018-10-12 23:27:23,280 com.happypathway.tfe.Run: RunValidator: does not have attribute: message
2018-10-12 23:27:23,281 com.happypathway.tfe.Run: RunValidator: does not have attribute: source
2018-10-12 23:27:23,283 com.happypathway.tfe.Run: can't set attribute
2018-10-12 23:27:23,284 com.happypathway.tfe.Run: RunValidator: does not have attribute: status-timestamps
2018-10-12 23:27:23,286 com.happypathway.tfe.Run: RunValidator: does not have attribute: created-at
2018-10-12 23:27:23,288 com.happypathway.tfe.Run: RunValidator: does not have attribute: canceled-at
2018-10-12 23:27:23,289 com.happ

In [27]:
run.get()
if run.status == "planned":
    run.apply()
run.status

2018-10-12 23:24:57,195 com.happypathway.tfe.Run: RunValidator: does not have attribute: is-destroy
2018-10-12 23:24:57,196 com.happypathway.tfe.Run: RunValidator: does not have attribute: message
2018-10-12 23:24:57,197 com.happypathway.tfe.Run: RunValidator: does not have attribute: source
2018-10-12 23:24:57,198 com.happypathway.tfe.Run: can't set attribute
2018-10-12 23:24:57,200 com.happypathway.tfe.Run: RunValidator: does not have attribute: status-timestamps
2018-10-12 23:24:57,202 com.happypathway.tfe.Run: RunValidator: does not have attribute: created-at
2018-10-12 23:24:57,204 com.happypathway.tfe.Run: RunValidator: does not have attribute: canceled-at
2018-10-12 23:24:57,205 com.happypathway.tfe.Run: RunValidator: does not have attribute: has-changes
2018-10-12 23:24:57,207 com.happypathway.tfe.Run: RunValidator: does not have attribute: actions
2018-10-12 23:24:57,208 com.happypathway.tfe.Run: RunValidator: does not have attribute: plan-only
2018-10-12 23:24:57,209 com.happ

'errored'